<a href="https://colab.research.google.com/github/FlowAlpha/econ123/blob/main/proj2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2

## Setup

In [ ]:
# Setup

setwd("~/Documents/ECON123/Proj2")
library(haven)
library(ggplot2)
library(dplyr)
library(tseries)
library(cowplot)
library(car)
library(targeted)
library(pwr)

In [ ]:
# Import data
dta <- read_dta("data/AEJApp-20090168_data.dta")
write.csv(dta, file = "data/AEJApp-data.csv")
dt <- read.csv("data/AEJApp-data.csv")

# Globals
# Get Women/Men treated and non-treated
sel <- filter(dt, select==1)
nsel <- filter(dt, select==0)
dt_f <- filter(dt, dwomen==1)
dt_m <- filter(dt, dwomen==0)


## Questions

### Question 1
    What defines the power of the experiment: the proportion randomized into treatment and control (2/3 and 1/3) or the proportion sampled in treatment and control (1/2 and 1/2)? 


Power refers to the sample distributions, so the propertion sampled would define the power of the experiment. 

### Question 2
    Examine how well balanced the resulting treatment and control samples are. Do this first by comparing the means of selected variables between treatment and control. Construct individual t-tests for the null hypothesis that they are the same across treatment and control at baseline. Then construct a joint F-test of the null hypothesis that a selected set of baseline characteristics cannot explain allocation to treatment. What reasoning did you use to choose the variables to test for balance? 

In [ ]:
# Get Women/Men treated and non-treated
sel <- filter(dt, select==1)
nsel <- filter(dt, select==0)


# Calculate means of above
sel_means <- as.double(colMeans(sel, na.rm=TRUE))
nsel_means <- as.double(colMeans(nsel, na.rm=TRUE))

# Create dataframe with information, reduce to variables we care about
var_names <- c('Age', 'Married', 'Employment', 'Salary', 'Tenure', 'Days Worked', 'Hours Worked', 'Contracted', 'Formal', 'Yrs Education', 'Paid Employment')
df <- data.frame(colnames(dt), sel_means, nsel_means)
colnames(df) <- c('var', 'Treated_means', 'Control_means')
df <- filter(df, var =="empl_04" | var == "pempl_04" | var == "contract_04" | var == "salary_04" | var == "dformal_04" | var == "tenure_04" | var == "days_04" | var == "hours_04" | var == "educ_lb" | var == "age_lb" | var == "dmarried_lb")

# Make it easy to read
df$var <- var_names
df$diff <- round(df$Treated_means-df$Control_means, digits=3)
df[,-1] <- round(df[,-1], digits=3)
means1_df <- df
print(means1_df)

               var Treated_means Control_means     diff
1              Age        21.079        21.234   -0.155
2          Married         0.186         0.201   -0.016
3       Employment         0.535         0.501    0.035
4           Salary    105722.451     99662.552 6059.899
5           Tenure         4.034         3.201    0.833
6      Days Worked        12.761        12.042    0.719
7     Hours Worked        26.785        24.882    1.903
8       Contracted         0.084         0.085   -0.001
9           Formal         0.097         0.085    0.013
10   Yrs Education        10.172         9.993    0.179
11 Paid Employment         0.388         0.346    0.043


In [ ]:
# t-testing and f-testing

ttest <- t.test(means1_df[2], means1_df[3])
ttest$p.value

# F-testing with standout variables above

model <- lm(select ~ salary_04 + hours_04 + days_04 + empl_04, data=dt)
model1 <- lm(select ~ empl_04 + pempl_04 + contract_04 + salary_04 + dformal_04 + tenure_04 + days_04 + hours_04 + educ_lb + age_lb + dmarried_lb, data=dt)
test<- linearHypothesis(model, c("salary_04=0", "hours_04=0", "days_04=0", "empl_04=0"))
test2 <- var.test(means1_df$Treated_means, means1_df$Control_means)
#test3 <- var.test(sel$pempl_04, nsel$pempl_04)
test2$p.value
test$'Pr(>F)'[2]
#t<-aov(lm(select ~ empl_04 + pempl_04 + contract_04 + salary_04 + dformal_04 + tenure_04 + days_04 + hours_04 + educ_lb + age_lb + dmarried_lb, data=dt))
#summary(t)

[1] 0.9671246

[1] 0.8555745

[1] 0.1683051

* 1st number represents t-test p-value
* 2nd number represents the p-value on variation f-test on the means
* 3rd number represents the p-value on linear hypothesis on the null hypothesis that coefficients that salary, hours worked, and days worked are zero when they are regressed on select (treatment variable).

Above, used the var.test to f-test the variances in the means of many variables (most of the ones shown in the first paper). The p-value is 0.856 > 0.05, therefore there is no significant difference in the variances of the means. Then, using the linearHypothesis to test a few of the variables. Using some of the variables from the difference table and linearHypothesis (uses F-test), we can see that when salary, hours, days, and employment are not enought to reject the null hypothesis. Logically, the alternative hypothesis is excepted that those variables statisically cannot explain allocation to treatment.

### Question 3
      Estimate the treatment effect for employment and earnings for men and women separately and overall.



In [ ]:
# Treatment effect for employment overall
E_Y_X_1 <- dt %>% filter(select==1) %>% summarise(conditional_mean = mean(empl_06,na.rm=TRUE))
E_Y_X_0 <- dt %>% filter(select==0) %>% summarise(conditional_mean = mean(empl_06, na.rm=TRUE))
ATE_e_both <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

E_Y_X_1 <- dt %>% filter(select==1) %>% summarise(conditional_mean = mean(salary_06,na.rm=TRUE))
E_Y_X_0 <- dt %>% filter(select==0) %>% summarise(conditional_mean = mean(salary_06, na.rm=TRUE))
ATE_s_both <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Women
E_Y_X_1 <- dt_f %>% filter(select==1) %>% summarise(conditional_mean = mean(empl_06,na.rm=TRUE))
E_Y_X_0 <- dt_f %>% filter(select==0) %>% summarise(conditional_mean = mean(empl_06, na.rm=TRUE))
ATE_e_f <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

E_Y_X_1 <- dt_f %>% filter(select==1) %>% summarise(conditional_mean = mean(salary_06,na.rm=TRUE))
E_Y_X_0 <- dt_f %>% filter(select==0) %>% summarise(conditional_mean = mean(salary_06, na.rm=TRUE))
ATE_s_f <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Men
E_Y_X_1 <- dt_m %>% filter(select==1) %>% summarise(conditional_mean = mean(empl_06,na.rm=TRUE))
E_Y_X_0 <- dt_m %>% filter(select==0) %>% summarise(conditional_mean = mean(empl_06, na.rm=TRUE))
ATE_e_m <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

E_Y_X_1 <- dt_m %>% filter(select==1) %>% summarise(conditional_mean = mean(salary_06,na.rm=TRUE))
E_Y_X_0 <- dt_m %>% filter(select==0) %>% summarise(conditional_mean = mean(salary_06, na.rm=TRUE))
ATE_s_m <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# All together:
ATE <- c(ATE_e_both, ATE_e_f, ATE_e_m, ATE_s_both, ATE_s_f, ATE_s_m)
group <- c('Employment Both', 'Employment (f)', 'Employment (m)', 'Salary Both', 'Salary (f)', 'Salary (m)')


# ATT effect for employment overall
E_Y_X_1 <- sel %>% filter(select==1) %>% summarise(conditional_mean = mean(empl_06,na.rm=TRUE))
E_Y_X_0 <- nsel %>% filter(select==0) %>% summarise(conditional_mean = mean(empl_06, na.rm=TRUE))
ATT_e_both <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

E_Y_X_1 <- sel %>% filter(select==1) %>% summarise(conditional_mean = mean(salary_06,na.rm=TRUE))
E_Y_X_0 <- nsel %>% filter(select==0) %>% summarise(conditional_mean = mean(salary_06, na.rm=TRUE))
ATT_s_both <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Women
E_Y_X_1 <- filter(sel, dwomen==1) %>% filter(select==1) %>% summarise(conditional_mean = mean(empl_06,na.rm=TRUE))
E_Y_X_0 <- filter(nsel, dwomen==1) %>% filter(select==0) %>% summarise(conditional_mean = mean(empl_06, na.rm=TRUE))
ATT_e_f <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

E_Y_X_1 <- filter(sel, dwomen==1) %>% filter(select==1) %>% summarise(conditional_mean = mean(salary_06,na.rm=TRUE))
E_Y_X_0 <- filter(nsel, dwomen==1) %>% filter(select==0) %>% summarise(conditional_mean = mean(salary_06, na.rm=TRUE))
ATT_s_f <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Men
E_Y_X_1 <- filter(sel, dwomen==0) %>% filter(select==1) %>% summarise(conditional_mean = mean(empl_06,na.rm=TRUE))
E_Y_X_0 <- filter(nsel, dwomen==0) %>% filter(select==0) %>% summarise(conditional_mean = mean(empl_06, na.rm=TRUE))
ATT_e_m <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

E_Y_X_1 <- filter(sel, dwomen==0) %>% filter(select==1) %>% summarise(conditional_mean = mean(salary_06,na.rm=TRUE))
E_Y_X_0 <- filter(nsel, dwomen==0) %>% filter(select==0) %>% summarise(conditional_mean = mean(salary_06, na.rm=TRUE))
ATT_s_m <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Together:
ATT <- c(ATT_e_both, ATT_e_f, ATT_e_m, ATT_s_both, ATT_s_f, ATT_s_m)
treat_frame <- data.frame(group, ATE, ATT)
print(treat_frame)


            group       ATE       ATT
1 Employment Both     0.038     0.038
2  Employment (f)     0.066     0.066
3  Employment (m)    -0.006    -0.006
4     Salary Both 39851.191 39851.191
5      Salary (f) 37462.658 37462.658
6      Salary (m) 37123.674 37123.674


It's important to note that these ATEs ATTs are biased because the randomization took place within the control centers, not before being assigned.

### Question 4
    Test that the impact on males and females is the same. 

First, we'll do what we did in #2, but separating by gender:

In [ ]:
# Get Women/Men treated and non-treated
sel <- filter(dt, select==1)
nsel <- filter(dt, select==0)
fsel<- filter(sel, dwomen == 1)
msel<- filter(sel, dwomen == 0)
f_nsel <- filter(nsel, dwomen==1)
m_nsel<- filter(nsel, dwomen == 0)

# Calculate means of above
fs_means <- as.double(colMeans(fsel, na.rm=TRUE))
ms_means <- as.double(colMeans(msel, na.rm=TRUE))
fns_means <- as.double(colMeans(f_nsel, na.rm=TRUE))
mns_means <- as.double(colMeans(m_nsel, na.rm=TRUE))

# Create dataframe with information, reduce to variables we care about
var_names <- c('Age', 'Married', 'Employment', 'Salary', 'Tenure', 'Days Worked', 'Hours Worked', 'Contracted', 'Formal', 'Yrs Education', 'Paid Employment')
df <- data.frame(colnames(dt), fs_means, ms_means, fns_means, mns_means)
colnames(df) <- c('var', 'fs_means', 'ms_means','fns_means', 'mns_means')
df <- filter(df, var =="empl_04" | var == "pempl_04" | var == "contract_04" | var == "salary_04" | var == "dformal_04" | var == "tenure_04" | var == "days_04" | var == "hours_04" | var == "educ_lb" | var == "age_lb" | var == "dmarried_lb")

# Make it easy to read
df$var <- var_names
df$f_diff <- round(df$fs_means-df$fns_means, digits=3)
df$m_diff <- round(df$ms_means-df$mns_means, digits=3)
df$diff_of_diff <- round((df$fs_means-df$fns_means) - (df$ms_means-df$mns_means), digits=3)
df[,-1] <- round(df[,-1], digits=3)
means_df2 <- df[,c(1,2,4,6,3,5,7, 8)]

print(means_df2)


               var  fs_means fns_means   f_diff   ms_means  mns_means   m_diff
1              Age    21.255    21.392   -0.138     20.880     21.043   -0.163
2          Married     0.270     0.262    0.008      0.091      0.128   -0.037
3       Employment     0.474     0.455    0.018      0.605      0.555    0.050
4           Salary 88448.292 83923.286 4525.006 125270.951 118745.958 6524.992
5           Tenure     3.756     2.753    1.003      4.356      3.777    0.579
6      Days Worked    11.373    10.793    0.580     14.331     13.557    0.775
7     Hours Worked    23.511    22.132    1.378     30.490     28.216    2.274
8       Contracted     0.071     0.070    0.002      0.098      0.103   -0.005
9           Formal     0.078     0.064    0.014      0.119      0.110    0.009
10   Yrs Education    10.098     9.929    0.169     10.257     10.071    0.186
11 Paid Employment     0.352     0.327    0.025      0.429      0.368    0.061
   diff_of_diff
1         0.025
2         0.045
3   

* fs_means = Female treated mean
* fns_means = Female not treated mean
* f_diff = Difference (fs_means - fns_means)
* ms_means = Male treated mean
* mns_means = Male not treated mean
* m_diff Difference (ms_means - mns_means)
* diff_of_diff = f_diff - m_diff

Using the previous data (from 3):

In [ ]:
print(treat_frame)

            group       ATE       ATT
1 Employment Both     0.038     0.038
2  Employment (f)     0.066     0.066
3  Employment (m)    -0.006    -0.006
4     Salary Both 39851.191 39851.191
5      Salary (f) 37462.658 37462.658
6      Salary (m) 37123.674 37123.674


We can see that there is a difference in employment treatment effect and salary treatment effect between men and women. The impact is greater for women, especially for employment.

### Quetsion 5
    The randomization took place within each training center. However, if the proportion of treated and controls sampled from each center is the same for all centers this turns out to be equivalent to randomizing overall (i.e. we can ignore that different people belong to different training centers). Check whether this is true in your data by testing the hypothesis that the results for the treatment effect are the same whether you allow for center fixed effects or not. 

In [ ]:
# Employment
print("Employment")
casual_effect_e1 <- lm(empl_06 ~ select, data=dt)
course_casual_effect_e1<-lm(empl_06 ~ select + coursefixe, data=dt)

print(summary(casual_effect_e1)$coefficients)
print(summary(course_casual_effect_e1)$coefficients)
#summary(casual_effect)$coefficients[8]
print("---------------------------------------------------------")
# Income
print("Income")
casual_effect_i1 <- lm(salary_06 ~ select, data=dt)
course_casual_effect_i1<-lm(salary_06 ~ select + coursefixe, data=dt)

print(summary(casual_effect_i1)$coefficients)
print(summary(course_casual_effect_i1)$coefficients)

[1] "Employment"
              Estimate Std. Error   t value   Pr(>|t|)
(Intercept) 0.72305683 0.01116138 64.782010 0.00000000
select      0.03778725 0.01537446  2.457794 0.01403142
                 Estimate   Std. Error   t value     Pr(>|t|)
(Intercept)  0.8041962963 1.741426e-02 46.180342 0.000000e+00
select       0.0364552655 1.529222e-02  2.383910 1.718715e-02
coursefixe  -0.0003544017 5.860595e-05 -6.047197 1.642232e-09
[1] "---------------------------------------------------------"
[1] "Income"
             Estimate Std. Error   t value      Pr(>|t|)
(Intercept) 215786.43   5443.210 39.643234 1.738639e-280
select       39851.19   7497.853  5.315014  1.138801e-07
               Estimate Std. Error   t value      Pr(>|t|)
(Intercept) 228941.9522 8535.22690 26.823183 2.815375e-143
select       39635.2298 7495.15401  5.288114  1.317979e-07
coursefixe     -57.4608   28.72446 -2.000414  4.553908e-02


**Employment/Income**:
Above we can see that the coefficient for *select* hardly changes when *coursefixe* is added. The p-values are also < 0.05, indicating we can be fairly confident in this quick regression analysis.

### Question 6
    Provide some intuition of  what  the  center  fixed  effects  capture.  Compare the  R2 when  you  include  and  do  not  include  fixed  effects.  Remember  that the  R2 =  1-(residual  sum  of  squares)/(total  sum  of  squares).  Using  this relationship  rewrite  the  F-statistic  comparing  the  regression  with  and without fixed effects and test the null hypothesis that they are not important.  How  do  you  interpret  this  result?  Can  the  center  fixed  effects be  significant  themselves  but  not  matter for estimating the treatment effect? Explain. 

The center fixed effects allow us to do the above analysis. You need to account of these centers because the randomization occured *within* these centers. If you don't account for the course/center fixed effects, you essentially lose the ability to do bias analysis becase you lose the unit of randomization.

In [ ]:
# Employment
print("Employment")
casual_effect_e2 <- lm(empl_06 ~ select, data=dt)
course_casual_effect_e2<-lm(empl_06 ~ select + coursefixe, data=dt)
print("R^2")
print(summary(casual_effect_e2)$r.squared)
print(summary(course_casual_effect_e2)$r.squared)
print("F-stat")
print(summary(casual_effect_e2)$fstatistic[1])
print(summary(course_casual_effect_e2)$fstatistic[1])

print("----------------")

# Income
print("Income")
casual_effect_i2 <- lm(salary_06 ~ select, data=dt)
course_casual_effect_i2<-lm(salary_06 ~ select + coursefixe, data=dt)
print("R^2")
print(summary(casual_effect_i2)$r.squared)
print(summary(course_casual_effect_i2)$r.squared)
print("F-stat")
print(summary(casual_effect_i2)$fstatistic[1])
print(summary(course_casual_effect_i2)$fstatistic[1])

[1] "Employment"
[1] "R^2"
[1] 0.001863831
[1] 0.0130241
[1] "F-stat"
   value 
6.040752 
   value 
21.33788 
[1] "----------------"
[1] "Income"
[1] "R^2"
[1] 0.008656823
[1] 0.009881966
[1] "F-stat"
   value 
28.24937 
   value 
16.13862 
Linear hypothesis test

Hypothesis:
coursefixe = 0

Model 1: restricted model
Model 2: empl_06 ~ select + coursefixe

  Res.Df   RSS Df Sum of Sq      F    Pr(>F)    
1   3235 617.0                                  
2   3234 610.1  1    6.8988 36.569 1.642e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
Linear hypothesis test

Hypothesis:
coursefixe = 0

Model 1: restricted model
Model 2: salary_06 ~ select + coursefixe

  Res.Df        RSS Df  Sum of Sq      F  Pr(>F)  
1   3235 1.4674e+14                               
2   3234 1.4656e+14  1 1.8135e+11 4.0017 0.04554 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


* 1st value represents regression without the fixed effect
* 2nd value includes fixed effect

The slightly higher R^2 (especiallly for employment) isnt worrying because we don't expect treatment to explain all of employment (or even a large amount). 

Next, we can run hypothesis tests:

In [ ]:
test1<- linearHypothesis(course_casual_effect_e2, c("coursefixe=0"))
test2<- linearHypothesis(course_casual_effect_i2, c("coursefixe=0"))
print(test1)
print(test2)

Linear hypothesis test

Hypothesis:
coursefixe = 0

Model 1: restricted model
Model 2: empl_06 ~ select + coursefixe

  Res.Df   RSS Df Sum of Sq      F    Pr(>F)    
1   3235 617.0                                  
2   3234 610.1  1    6.8988 36.569 1.642e-09 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1
Linear hypothesis test

Hypothesis:
coursefixe = 0

Model 1: restricted model
Model 2: salary_06 ~ select + coursefixe

  Res.Df        RSS Df  Sum of Sq      F  Pr(>F)  
1   3235 1.4674e+14                               
2   3234 1.4656e+14  1 1.8135e+11 4.0017 0.04554 *
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


Though *coursefixe* for income has a larger p-value, they are both still significant enough to accept the hypothesis that *coursefixe* does not matter when estimating treatment effect. That doesn't mean that it can't be significant. 

### Question 7
    What  is  the  treatment  unit:  the  center  or  the  individual?  Do  you  need  to take  into account  of  clustering  when  computing  the  standard  errors? Explain your answer. 

The treatment unit is the individual because the individual is the one which a treatment is constant. However, because of the analysis we did in Question 5 (our data shows that we can actually treat our data as if we randomized all), we can treat the unit of randomization as the individual. If we found that Q5 showed the opposite, we would need to take into account clustering RCT for any SE analysis because we would need to cluster the centers to prevent selection bias in our treatment effects.

### Question 8
    Provide  an  intuition  at  to  why  such  a  program  could  affect  marriage. Estimate the effect of the program on marriage by gender. 

We'll look at the treatment effect on a created variabel *m_change* which is
* 1 if someone got married in period 2 and wasn't in period 1
* 0 if marriage didn't change
* -1 if someone was married in period 1 but not in period 2

Note: This shows a certain effect, that of change in marriage which I think is more important than marriage itself. A better way to do just marriage is to calculate the difference in difference in marriage in the two period and account for that in the calculation. 

In [ ]:
# Treatment effect for employment overall
dt$m_change <- dt$dmarried_s-dt$dmarried_lb
E_Y_X_1 <- dt %>% filter(select==1) %>% summarise(conditional_mean = mean(m_change,na.rm=TRUE))
E_Y_X_0 <- dt %>% filter(select==0) %>% summarise(conditional_mean = mean(m_change, na.rm=TRUE))
ATE_both <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Women
dt_f$m_change <- dt_f$dmarried_s-dt_f$dmarried_lb
E_Y_X_1 <- dt_f %>% filter(select==1) %>% summarise(conditional_mean = mean(m_change,na.rm=TRUE))
E_Y_X_0 <- dt_f %>% filter(select==0) %>% summarise(conditional_mean = mean(m_change, na.rm=TRUE))
ATE_f <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Men
dt_m$m_change <- dt_m$dmarried_s-dt_m$dmarried_lb
E_Y_X_1 <- dt_m %>% filter(select==1) %>% summarise(conditional_mean = mean(m_change,na.rm=TRUE))
E_Y_X_0 <- dt_m %>% filter(select==0) %>% summarise(conditional_mean = mean(m_change, na.rm=TRUE))
ATE_m <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)


# All together:
ATE <- c(ATE_both, ATE_f, ATE_m)
Group_Change <- c('Together', 'Female', 'Male')
print(data.frame(Group_Change, ATE))

  Group_Change   ATE
1     Together 0.009
2       Female 0.004
3         Male 0.015


Above we can see that the estimated treatment made it more likely that men would have a "positive" change in marriage status. 

### Question 9
    Explore  the  effect  of  treatment  on  any  other  outcome  variable  of  your choice by gender. 

Tenure difference (period 2 - period 1)! There's some loss for everyone treated (6 months, 0.5) which has to be accounted for

In [ ]:
# Treatment effect for employment overall
dt$tenure_diff <- dt$tenure_06-dt$tenure_04
E_Y_X_1 <- dt %>% filter(select==1) %>% summarise(conditional_mean = mean(tenure_diff,na.rm=TRUE))
E_Y_X_0 <- dt %>% filter(select==0) %>% summarise(conditional_mean = mean(tenure_diff+0.5, na.rm=TRUE))
ATE_both <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Women
dt_f$tenure_diff <- dt_f$tenure_06-dt_f$tenure_04
E_Y_X_1 <- dt_f %>% filter(select==1) %>% summarise(conditional_mean = mean(tenure_diff,na.rm=TRUE))
E_Y_X_0 <- dt_f %>% filter(select==0) %>% summarise(conditional_mean = mean(tenure_diff+0.5, na.rm=TRUE))
ATE_f <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)

# Men
dt_m$tenure_diff <- dt_m$tenure_06-dt_m$tenure_04
E_Y_X_1 <- dt_m %>% filter(select==1) %>% summarise(conditional_mean = mean(tenure_diff,na.rm=TRUE))
E_Y_X_0 <- dt_m %>% filter(select==0) %>% summarise(conditional_mean = mean(tenure_diff+0.5, na.rm=TRUE))
ATE_m <- round(as.double(E_Y_X_1 - E_Y_X_0), digits=3)


# All together:
ATE <- c(ATE_both, ATE_f, ATE_m)
Group_Change <- c('Together', 'Female', 'Male')
print(data.frame(Group_Change, ATE))

  Group_Change    ATE
1     Together -2.393
2       Female -1.607
3         Male -3.546


It's probably expected that the average treatment of tenure is negative. This indicates that people probably switch jobs. What is interesting is that there is a meaningful difference between men and women. We can look at the mean values below:

In [ ]:
print(mean(dt_f$tenure_06, na.rm=TRUE) - mean(dt_f$tenure_04, na.rm=TRUE))
print(mean(dt_m$tenure_06, na.rm=TRUE) - mean(dt_m$tenure_04, na.rm=TRUE))

[1] 3.205764
[1] 5.223993


Above shows that the increase in average tenure is less for women than for men across the board. 

In [ ]:
print(mean(filter(dt_f, select==1)$tenure_06, na.rm=TRUE) - mean(dt_f$tenure_04, na.rm=TRUE))
print(mean(filter(dt_m,select==1)$tenure_06, na.rm=TRUE) - mean(dt_m$tenure_04, na.rm=TRUE))

[1] 2.573673
[1] 3.767843


Above shows that the increase in average tenure is less for women than for men when treated.

The treatment analysis would suggest that "treated" women are more likely to stay at the same job and/or have greater tenure than they did before treatment (greater increase in tenure) when compared to men. Cool.

### Question 10
    Redo  the  power  calculations  and  derive  the  sample  size  you  would  need to detect an effect of 3 percentage points on employment when the size of the test you use is 5% and the required power is 80%. What would it taketo increase power to 90%? What sample size would you need for a power of 90% when using a significance level of 1%?

In [ ]:
power<-pwr.2p.test(h = ES.h(p1 = mean(sel$empl_06, na.rm=TRUE), p2 = mean(nsel$empl_06, na.rm=TRUE)), sig.level = 0.05, power = .80)

power2<-pwr.2p.test(h = ES.h(p1 = 0.1, p2 = 0.13), sig.level = 0.05, power = .80)
#plot(power2)
power3<-pwr.2p.test(h = ES.h(p1 = 0.1, p2 = 0.13), sig.level = 0.05, power = .90)
#plot(power3)
power2$n
power3$n-power2$n

[1] 1768.095

[1] 598.8876

Above we can see we would need n=1769 to detect the 3 percentage point effect and 599 more to increase that power from 80% to 90%

### Question 11
    Compare the results in the two papers and draw conclusions on the value of  the  intervention. Consider  the  cost-benefit  calculations  in  the  first paper and given the second paper discuss whether the assumptions made for the cost-benefit analysis in the first paper were accurate.

The results of the first paper showed postive treament effects for everyone's employment and women's earnings (overall, lots of improvments for women). This dataset was smaller than that used in the second paper which showed positive treatment effects for everyone's employment and earnings. The second paper, due to a lack of data on informal sectors, cannot do a comprehensive cost-benefit analysis. The first paper also has this problem, with the formal==0 meaning unemployment or informal sector. As such, the first paper should use the "conservative" analysis that the second does (assuming no employment growth in the informal sector) to establish a lower-bound.

### Question 12
    What  conclusions  does  the  first  experiment  reach  on  the  effectiveness  of the  program  for  men?  Are  these  modified  when  considering  the  follow up?  Explain.  Also  Consider  Figure  1  in  the  follow-up  paper  (page  136). Why are the figures on the left so much noisier than the ones on the right? 

The first experiment still sees the program being effective for men (though in fewer areas as women). Notably, income increased.

In the second paper, figure 1, the left figures are much noiser because they have been selected to give the experiment more signal. This increased signal could be the reason that the second paper found that the program was effective in more areas for men than the first paper

### Question 13
    Write  a  short  report  (300  words  approx.)  on  your  evaluation  of  this experiment. Within your report discuss possible issues relating to scaling up  the  program to  the  whole  economy  and  whether  the  information obtained  from  the  experiment  is  sufficient  for  making  the  decision  to adopt the program nationally. 

A good summary of the papers is that the first describes the initial results of a program, pending further robust research as given in the second paper. The first is a smaller dataset, the second is larger. The first (probably) has a weaker signal, the second has a stronger one. Overall, both show that this training program *is*, on average, beneficial to those who are able to take part. The results are conclusive enough to adopt the program on a greater scale. There would be new observations in locational fixtures, but the locations were taken into account in the datasets. The signal in some locations was likely not very high, but the overall results seem to show an improvement with treatment. A problem that the government would have to account for is volunteering. The papers describe that similar programs often have difficulty applying treatment to a randomization of the population because certain people (especially women) do not sign up for treatment even though it is shown to help. A solution could be to... force treatment, but that isn't even a perfect solution because that wouldn't guarantee the same results as the randomized trails in the paper (forced treatment effect != volunteered treatment effect). A better, harder, solution would be to initiate education/opportunity programs across the country alongside the ability to train. There are likely many psychological factors that could help improve volunteer randomization. Additionally, it would probably be wise to establish certain criteria for people to get treatment/training to maximize cost-benefit (ex: work a certain amount of years before training). This could be tuned to maximize cost-benefit for (an assumed) randomized treated individual. 

In summary, **is the information obtained  from  the  experiment  is  sufficient  for  making  the  decision  to adopt the program nationally**? 

Yes and no. Yes because the results are very *promising* and highly suggest that the program should be implemented. No/maybe if treatment allocation isn't handled correctly.